In [18]:
import json

import numpy as np
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from transformers import pipeline

In [11]:
hellaswag = load_dataset("hellaswag")
gsm = load_dataset("gsm8k", "main")
wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")

In [12]:
gsm_texts = [item["question"] + " " + item["answer"] for item in gsm["train"]]
hellaswag_texts = [f"{item['ctx']}{item['endings'][int(item['label'])]}" for item in hellaswag["train"]][:3000]
wikitext_texts = [item["text"] for item in wikitext["train"]]
wikitext_texts = wikitext_texts[:len(wikitext_texts) // 2]

# Removing new lines from texts
gsm_texts = [text.replace("\n", " ") for text in gsm_texts]
wikitext_texts = [text.replace("\n", " ") for text in wikitext_texts]
hellaswag_texts = [text.replace("\n", " ") for text in hellaswag_texts]

In [13]:
print("hellaswag", len(hellaswag_texts))
print("gsm", len(gsm_texts))
print("wikitext", len(wikitext_texts))

hellaswag 3000
gsm 7473
wikitext 18359


In [14]:
concatenated_texts = []
for i in range(6):
    st, en = i / 6 * len(wikitext_texts), (i + 1) / 6 * len(wikitext_texts)
    concatenated_texts += wikitext_texts[int(st):int(en)]

    st, en = i / 6 * len(hellaswag_texts), (i + 1) / 6 * len(hellaswag_texts)
    concatenated_texts += hellaswag_texts[int(st):int(en)]

    st, en = i / 6 * len(gsm_texts), (i + 1) / 6 * len(gsm_texts)
    concatenated_texts += gsm_texts[int(st):int(en)]

In [15]:
with open("imagination_dataset.txt", "w") as f:
    for text in concatenated_texts:
        f.write(text + "\n")

In [16]:
with open("wikitext.txt", "w") as f:
    for text in wikitext_texts:
        f.write(text + "\n")

In [17]:
! wc -w imagination_dataset.txt

1848547 imagination_dataset.txt


In [19]:
kt_full = json.load(open("kt_dataset_gpt_mini.json"))

Using only half of the KT_test

In [31]:
length = len(kt_full)
length = length // 2
length += (5 - length % 5)
length -= 1

In [30]:
kt_full[length - 1]

{'context': 'Humpty Dumpty has been adapted in various forms of media, including music and literature over the decades.',
 'sentence_true': 'His character is mentioned in songs by artists such as Hank Thompson and Aretha Franklin.',
 'sentence_false': 'His character is mentioned in songs only by instrumentalists like Ornette Coleman and Chick Corea.',
 'location': 17914}

Humpty Dumpty is the last topic in the imagination dataset

In [34]:
# Saving the new KT_dataset
with open("kt_dataset_gpt_mini_imagination.json", "w") as f:
    json.dump(kt_full[:length], f, indent=4)